In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [5]:
from pyspark.context import SparkContext 
sc = SparkContext.getOrCreate()
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate('local')
spark = SparkSession(sc)

In [6]:
import os
print os.getcwd()


/Users/bharvadl/Desktop/CourseWork/Search/project


In [7]:
import pandas as pd

In [8]:
##Toy Dataset code

df_rev = pd.read_csv("review.csv")

# lines = spark.read.csv(mpath+"review.csv").rdd
# parts = lines.map(lambda row: row.value.split("::"))

In [33]:
print df_rev
df_rev = df_rev.sample(frac=0.4)
#ratings = spark.createDataFrame(df_rev)

         funny                 user_id               review_id  \
2578081      0  u9riWTMe_E-jb93IFLsqCA  QikQkhQQAQgdOZeGFBCLEg   
1802593      1  XOaE6wGOiHi-PvDjdzRDIw  Tuym_R2uysY-2pko8iWPXg   
2795519      1  D0_EDhZvO5z3UEPszmxiYg  z1VKPRuPdwMDMFduIsQrzg   
1736492      0  vnWKs2dgqDuR_yNI5qJG5g  ahF7oKprvUYv3At5gEQ45w   
2844288      0  gAmmEZLSdj5KXvOlPUQRfQ  5mxrOBcWMp7lkQ2EUzZwLg   
3995783      0  uPdQLnVf0wDwNW4ScHnA2Q  UGdGtw4WxFgz4LRECY-tLA   
2545823      0  RsuNc8EdA9J_dsImcheaRQ  Z_5bB4QSofAxngMT3GwZKA   
2858864      0  dENG4JUCCJwsr_ErLNJ2tA  V_5vVcsVleYqXl4hMxrMAw   
4679757      1  ZEq0WtRJD9Bl_vYgCsbfOg  tZWsCtDS1I5Se3rmN33qYg   
2101651      0  B9vCYb4mfX5ba1KlxsDy1w  jo_c_teASwhXfOuDEiHsJQ   
3917059      0  ZZc-pFlOH85eRoUqbtz2Nw  UZPVaC4NeSRV2oFRd34tNw   
2609487      0  X_ATCvgV1rHzPaDqGLI9kw  tT7UhoQMWc1a9rzryLDMpA   
3695531      1  Afo2N4GUlYsmYM9KcbW7IQ  3W8dQSLK4bD-VzzpwQjmmQ   
2262743      1  yZ7hMvKPi05tre3gYdhKHQ  yyaUM-5cPa3tQKwXnPZZbQ   
2674423   

In [15]:
df_new = df_rev[['user_id','business_id','stars']]
df_new = pd.DataFrame(df_new.groupby(['user_id','business_id']).agg('mean'))
df_new.reset_index(inplace=True)
df_new.head()

,user_id,business_id,stars
0,---1lKK3aKOuomHnwAkAow,1Vn_lex3LGGwuTo-xeJnww,5
1,---1lKK3aKOuomHnwAkAow,DV13F0bhe55dV1AhwoO50g,5
2,---1lKK3aKOuomHnwAkAow,HZdtHOEaKUL2SlWj5owgCA,5
3,---1lKK3aKOuomHnwAkAow,Ks0M3J4vZAKsHPuCINz5fQ,4
4,---1lKK3aKOuomHnwAkAow,NAuUGBenC_TJoj2c6y9rbg,5


In [37]:
def gen_num(vals):
    return int("".join([str(ord(val)) for val in vals]))
df_new['user_id_int'] = map(gen_num,df_new["user_id"])
print df_new["user_id_int"]

0         4545454910875755197757911711110972110119651076...
1         4545454910875755197757911711110972110119651076...
2         4545454910875755197757911711110972110119651076...
3         4545454910875755197757911711110972110119651076...
4         4545454910875755197757911711110972110119651076...
5         4545454910875755197757911711110972110119651076...
6         4545454910875755197757911711110972110119651076...
7         4545454910875755197757911711110972110119651076...
8         4545454910875755197757911711110972110119651076...
9         4545454910875755197757911711110972110119651076...
10        4545454910875755197757911711110972110119651076...
11        4545454910875755197757911711110972110119651076...
12        4545454910875755197757911711110972110119651076...
13        4545454910875755197757911711110972110119651076...
14        4545454910875755197757911711110972110119651076...
15        4545481071171177610911789661015182109117487312...
16        454548107117117761091178966101

In [29]:
ratings = spark.createDataFrame(df_new)
print df_new,ratings

                       user_id             business_id  stars  user_id_int  \
0       ---1lKK3aKOuomHnwAkAow  1Vn_lex3LGGwuTo-xeJnww      5            0   
1       ---1lKK3aKOuomHnwAkAow  DV13F0bhe55dV1AhwoO50g      5            1   
2       ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA      5            2   
3       ---1lKK3aKOuomHnwAkAow  Ks0M3J4vZAKsHPuCINz5fQ      4            3   
4       ---1lKK3aKOuomHnwAkAow  NAuUGBenC_TJoj2c6y9rbg      5            4   
5       ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g      2            5   
6       ---1lKK3aKOuomHnwAkAow  R-McIj4Psxl1VlEacsXeRg      4            6   
7       ---1lKK3aKOuomHnwAkAow  RhTBGAHFqnFTgSUDJtBuIQ      5            7   
8       ---1lKK3aKOuomHnwAkAow  eduRavkml8awmPachSZXuw      1            8   
9       ---1lKK3aKOuomHnwAkAow  jGv6tpmE7F9CG5qmLHad6A      5            9   
10      ---1lKK3aKOuomHnwAkAow  jobP3ywRd3QNZ_GCoPG2DQ      3           10   
11      ---1lKK3aKOuomHnwAkAow  nv2z5SYQR0JlzflUrm00Mw      5   

In [32]:

# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=long(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)

# (training, test) = ratings.randomSplit([0.8, 0.2])

# # Build the recommendation model using ALS on the training data
# # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# als = ALS(maxIter=5, regParam=0.01, userCol="user_id_int", itemCol="business_id_int", ratingCol="stars")
# model = als.fit(training)

# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
#                                 predictionCol="prediction")
# print predictions
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

# # # Generate top 10 movie recommendations for each user
# # userRecs = model.recommendForAllUsers(10)
# # # Generate top 10 user recommendations for each movie
# # movieRecs = model.recommendForAllItems(10)

DataFrame[user_id: string, business_id: string, stars: bigint, user_id_int: bigint, business_id_int: bigint]                        user_id             business_id  stars  user_id_int  \
0       ---1lKK3aKOuomHnwAkAow  1Vn_lex3LGGwuTo-xeJnww      5            0   
1       ---1lKK3aKOuomHnwAkAow  DV13F0bhe55dV1AhwoO50g      5            1   
2       ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA      5            2   
3       ---1lKK3aKOuomHnwAkAow  Ks0M3J4vZAKsHPuCINz5fQ      4            3   
4       ---1lKK3aKOuomHnwAkAow  NAuUGBenC_TJoj2c6y9rbg      5            4   
5       ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g      2            5   
6       ---1lKK3aKOuomHnwAkAow  R-McIj4Psxl1VlEacsXeRg      4            6   
7       ---1lKK3aKOuomHnwAkAow  RhTBGAHFqnFTgSUDJtBuIQ      5            7   
8       ---1lKK3aKOuomHnwAkAow  eduRavkml8awmPachSZXuw      1            8   
9       ---1lKK3aKOuomHnwAkAow  jGv6tpmE7F9CG5qmLHad6A      5            9   
10      ---1lKK3aKOuomHnwAkAow  j